In [ ]:
import laspy
import math
import pandas as pd
from tabulate import tabulate
import numpy as np



    
    

# def get_coordinate_pairs(df, center = (0, 0)):    
#     coordinates = [center]
#     for index, row in df.iterrows():
#         coordinates.append((row['X_distance'], row['Y_distance']))
        
#     return coordinates
    



# def get_furthest_center_coordinates(coordinate_pairs):
#     max_distance = 0
#     furthest_coordinates = ()
#     furthest_indices = ()
#     # Calculate distances between all pairs of coordinates
#     for i in range(len(coordinate_pairs)):
#         for j in range(i + 1, len(coordinate_pairs)):
#             distance = calculate_distance(*coordinate_pairs[i], *coordinate_pairs[j])

#             # Update furthest_coordinates if a new maximum distance is found
#             if distance > max_distance:
#                 # print(f"Distance between point {i + 1} and point {j + 1}: {distance}")
#                 max_distance = distance
#                 furthest_coordinates = (coordinate_pairs[i], coordinate_pairs[j])
#                 furthest_indices = (i, j-1)

#     return furthest_coordinates, furthest_indices


# def get_info_from_df(df, indices):
    
#     selected_rows = [df.loc[index] for index in indices]
#     discovered_df = pd.concat(selected_rows, axis=1).T.reset_index(drop=True)
#     discovered_df.insert(0, 'Index', indices)
    
#     print("discovered_df:")
#     print(tabulate(discovered_df, headers='keys', tablefmt='fancy_grid'))
    
#     discovered_csv_file_path = r'C:\Users\JooHyunAhn\Interpine\DataSets\TreeTools_PlayGroundSet\Straightess_issue\SelectedSampleData\Plot95_18_disc.csv'
#     discovered_df.to_csv(discovered_csv_file_path)
    
#     return discovered_df


def read_csv(csv_file_path):
    df = pd.read_csv(csv_file_path, sep=',')
    
    return df

# Function to calculate distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def data_derivation(df):
    bottom_center_x = 0
    bottom_center_y = 0
    df['Relocate_X'] = df['Bin_X'] - df['Base_X']
    df['Relocate_Y'] = df['Bin_Y'] - df['Base_Y']
    df['Distance_From_Center'] = ((df['Relocate_X'] - bottom_center_x)**2 + (df['Relocate_Y'] - bottom_center_y)**2).apply(math.sqrt)
    
    df['Radius'] = df['MA_Diameter'] / 2
    
    return df

def get_furthest_cylinders(df):
    
    df_sorted = df.sort_values(by='Height_Bin')        
    max_distance_index = df_sorted['Distance_From_Center'].idxmax()
    max_distance_row = df_sorted.loc[max_distance_index].to_frame().T
    
    bottom_cylinder_row = df.nsmallest(1, 'Height_Bin')
    
    # Concatenate DataFrames with ignore_index set to False
    discovered_df = pd.concat([max_distance_row, bottom_cylinder_row], ignore_index=False)
    discovered_sorted_df = discovered_df.sort_values(by='Height_Bin')
    bottom_xy = (discovered_sorted_df.loc[0, 'Relocate_X'], discovered_sorted_df.loc[0, 'Relocate_Y'])
    furthest_xy = (discovered_sorted_df.loc[1, 'Relocate_X'], discovered_sorted_df.loc[1, 'Relocate_Y'])
    two_coordinates = [bottom_xy, furthest_xy]
    print(two_coordinates)
    return discovered_sorted_df, two_coordinates

def get_length_hypotenuse_xy(two_coordinates):
    print(two_coordinates[0])
    hypotenuse = calculate_distance(*two_coordinates[0], *two_coordinates[-1])
    x_length = abs(two_coordinates[-1][0] - two_coordinates[0][0])
    y_length = abs(two_coordinates[-1][-1] - two_coordinates[0][-1])
    
    return hypotenuse, x_length, y_length    
    
def get_theta(hypotenuse, x_length):
    cosine_theta = x_length / hypotenuse
    theta_alpha = math.degrees(math.acos(cosine_theta))
    theta_beta = 90 - theta_alpha
    
    return theta_alpha, theta_beta

def get_furthest_circumference_coordinates(df):
    
    # Order by 'Height_Bin'
    df_sorted = df.sort_values(by='Height_Bin')
    
    df_bottom_cyl_info = df_sorted.head(1)
    df_top_cyl_info = df_sorted.tail(1)

    if df_top_cyl_info['X_distance'] < df_bottom_cyl_info['X_distance']:
        direction = 'left'
    else:
        direction = 'right'
    
    # df_sorted['Direction'] = 
    
        
    
    print(tabulate(df_bottom_cyl_info, headers='keys', tablefmt='fancy_grid'))
    print(tabulate(df_top_cyl_info, headers='keys', tablefmt='fancy_grid'))
    
    return 
    


def main():
    pd.set_option('display.max_colwidth', None) # this is just to check a dataframe easily
    csv_file_path = r'C:\Users\JooHyunAhn\Interpine\DataSets\TreeTools_PlayGroundSet\Straightess_issue\SelectedSampleData\Plot95_18.csv'
    # new_csv_file_path = r'C:\Users\JooHyunAhn\Interpine\DataSets\TreeTools_PlayGroundSet\Straightess_issue\SelectedSampleData\Plot95_18_new.csv'
    df = read_csv(csv_file_path)
    derivated_df = data_derivation(df)
    discovered_sorted_df, two_coordinates = get_furthest_cylinders(derivated_df)
    print(two_coordinates)
    # hypotenuse, x_length, y_length = get_length_hypotenuse_xy(two_coordinates)
    # print(hypotenuse, x_length, y_length)

    # print(tabulate(discovered_sorted_df, headers='keys', tablefmt='fancy_grid'))
    # print(tabulate(max_distance_row, headers='keys', tablefmt='fancy_grid'))
    # derivated_df.to_csv(new_csv_file_path)
    
    # coordinate_pairs = get_coordinate_pairs(new_df)
    # furthest_coordinates, furthest_indices = get_furthest_center_coordinates(coordinate_pairs)
    # hypotenuse, x_length, y_length = get_length_hypotenuse_xy(furthest_coordinates)
    # theta_alpha, theta_beta = get_theta(hypotenuse, x_length)
    # discovered_df = get_info_from_df(new_df, furthest_indices)
    # get_furthest_circumference_coordinates(discovered_df)
    
    # print("theta_alpha, theta_beta", theta_alpha, theta_beta)
    # print("hypotenuse", hypotenuse, "x_length", x_length, "y_length", y_length)
    # print(f"Furthest coordinates: {furthest_coordinates}")
    # print(f"Indices of furthest coordinates: {furthest_indices}")
    
 
main()

In [2]:
import math

# Coordinates of the points
a = (1, 1, 0)
b = (3, 3, 6)

# Calculate the Euclidean distance
distance = math.sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2 + (b[2] - a[2])**2)

print(f"The distance between points a and b is: {distance}")

The distance between points a and b is: 6.6332495807108


In [2]:
## Parametric equation
def point_on_line_at_height(start_coords, end_coords, target_height):
    x1, y1, z1 = start_coords
    x2, y2, z2 = end_coords
    
    # Solve for t when z = target_height
    t = (target_height - z1) / (z2 - z1)

    # Calculate the coordinates of the point on the line
    x = x1 + t * (x2 - x1)
    y = y1 + t * (y2 - y1)
    z = target_height
    
    return (x, y, z)

# Example usage:
top_coordinates = (0.3579305426172209, 0.3072399929457788, 9.1)
bottom_coordinates = (0.005236148773168993, 0.08984755281044242, 0.5)
target_height = 8.0

point_on_line = point_on_line_at_height(top_coordinates, bottom_coordinates, target_height)
print("Point on the line at height 8.0:", point_on_line)


Point on the line at height 8.0: (0.31281846898600496, 0.27943398316102647, 8.0)


In [3]:
import numpy as np

def find_point_along_line(a, b, distance_length):
    # Convert points to numpy arrays for easier vector operations
    a = np.array(a)
    b = np.array(b)

    # Calculate the direction vector
    direction_vector = b - a

    # Normalize the direction vector
    normalized_direction = direction_vector / np.linalg.norm(direction_vector)

    # Calculate the new point along the line at the desired distance
    new_point = a + distance_length * normalized_direction

    return tuple(new_point)

# Given points a and b
a = (0.31281846898600496, 0.27943398316102647, 8.0)
b = (-0.03777480089742226, 0.8618187571184339, 8.0)

# Given distance length
distance_length = 2.3788030921126895

# Find the point along the line from a at the specified distance
new_point = find_point_along_line(a, b, distance_length)
print("New point along the line:", new_point)


New point along the line: (-0.9140552998499649, 2.317444019050721, 8.0)


In [5]:
import numpy as np

def find_point_along_line(a, b, distance_length):
    # Convert points to numpy arrays for easier vector operations
    a = np.array(a)
    b = np.array(b)

    # Calculate the direction vector
    direction_vector = b - a

    # Normalize the direction vector
    normalized_direction = direction_vector / np.linalg.norm(direction_vector)

    # Calculate the new point along the line at the desired distance
    new_point = a + distance_length * normalized_direction

    return tuple(new_point)

# Given points a and b
a = (0.31281846898600496, 0.27943398316102647, 8.0)
b = (-0.03777480089742226, 0.8618187571184339, 8.0)

# Given distance length
distance_length = 2.3788030921126895

# Find the point along the line from a at the specified distance
new_point = find_point_along_line(a, b, distance_length)
print("New point along the line:", new_point)


New point along the line: (-0.9140552998499649, 2.317444019050721, 8.0)


In [6]:
import math

def get_theta_phi(hypotenuse, x_length):
    cosine_theta = x_length / hypotenuse
    theta_rad = math.acos(cosine_theta)
    phi_rad = math.asin(math.sqrt(1 - cosine_theta**2))

    return {'cosine_theta': cosine_theta, 'theta_rad': theta_rad, 'phi_rad': phi_rad}

# Example usage:
hypotenuse = 0.7562810323092551
x_length = 0.043999999994412065

result = get_theta_phi(hypotenuse, x_length)
print("Cosine Theta:", result['cosine_theta'])
print("Theta (in radians):", result['theta_rad'])
print("Phi (in radians):", result['phi_rad'])


Cosine Theta: 0.05817943081298881
Theta (in radians): 1.5125840244840945
Phi (in radians): 1.512584024484095


In [13]:
import math

def get_theta(hypotenuse, x_length):
    y_length = math.sqrt(hypotenuse**2 - x_length**2)
    
    cosine_theta = x_length / hypotenuse
    theta_degree = math.degrees(math.acos(cosine_theta))
    
    # Correct calculation of sine using sqrt(1 - cos^2(theta))
    sine_theta = math.degrees(math.sqrt(1 - cosine_theta**2))

    return {'cosine_theta': cosine_theta, 'theta_degree': theta_degree, 'sine_degree': sine_theta}

# Example usage:
hypotenuse = 0.7562810323092551
x_length = 0.043999999994412065

result = get_theta(hypotenuse, x_length)
print("Cosine Theta:", result['cosine_theta'])
print("Theta (in degrees):", result['theta_degree'])
print("Sine (in degrees):", result['sine_degree'])


Cosine Theta: 0.05817943081298881
Theta (in degrees): 86.66468076185139
Sine (in degrees): 57.19872861796811


In [14]:
import math

# Given values
x_length = 1.09999999999999
angle_degrees = 2.75812530305928

# Convert angle to radians
angle_radians = angle_degrees * math.pi / 180

# Calculate hypotenuse
hypotenuse = x_length / math.cos(angle_radians)

print("Hypotenuse:", hypotenuse)


Hypotenuse: 1.1012757481064575
